---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple
from random import random

In [2]:
@dataclass(frozen=True, order=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

# Nodes and Edges

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the attributes source and destination, but bidirectional edge cannot have these attributes. Which node should be a bidirectional edge's "source"? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional ones either. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we *can* build undirected graph with directional edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird and I want to avoid this.

`BaseEdge` is an abstract base class for edges. An edge has two nodes (`.node1` and `.node2`), can return them in a tuple (`.nodes`) and can be represented by a string where names of both nodes are separated by a separator. `BaseEdge` is also hashable.

`BaseEdge` also has `._separator` which is an abstract property to be determined by a concrete subclass. Its implementation depends on whether the edge is bidirectional or unidirectional.

(@ property means accessing `.nodes` on an instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.)

In [3]:
@dataclass(frozen=True, order=True)
class BaseEdge(ABC):
    """Abstract base class for immutable graph edge."""

    node1: Node
    node2: Node
    weight: int | float | None = None

    def __post_init__(self):
        # if weighted, then weight not None + bool operators lazy so this works
        if self.weighted and self.weight < 0:  # type: ignore
            raise ValueError("Weight cannot be negative")

    @property
    def weighted(self) -> bool:
        return self.weight is not None

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self.node1, self.node2

    @property
    @abstractmethod
    def _separator(self) -> str:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        sep = self._separator
        half = len(sep) // 2
        if self.weighted:
            return f"{node1} {sep[:half]}({self.weight}){sep[half:]} {node2}"
        else:
            return f"{node1} {self._separator} {node2}"


@dataclass(frozen=True, order=True)
class Edge(BaseEdge):
    """Bidirectional immutable graph edge."""

    def __post_init__(self):
        # Canonical ordering for dataclass's __eq__ and __hash__
        # to work correctly and for better (consistent) str representation
        if self.node1.name > self.node2.name:
            node1 = self.node1
            object.__setattr__(self, "node1", self.node2)
            object.__setattr__(self, "node2", node1)
        super().__post_init__()

    @property
    def _separator(self) -> str:
        return "<---->"


@dataclass(frozen=True, order=True)
class DirectedEdge(BaseEdge):
    """Unidirectional immutable graph edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float | None = None):
        # Improves signature of the constructor to reflect src and dest
        # Invariant: src=node1 and dest=node2
        super().__init__(src, dest, weight)

    def __repr__(self):
        # Reflects the renaming of node1 and node2 to src and dest
        return super().__repr__().replace("node1", "src").replace("node2", "dest")

    @property
    def src(self) -> Node:
        return self.nodes[0]

    @property
    def dest(self) -> Node:
        return self.nodes[1]

    @property
    def _separator(self) -> str:
        return "----->"

## Testing of Nodes and Edges

In [4]:
node1 = Node("1")
node2 = Node("2")

# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'

e12 = Edge(node1, node2)
we12_314 = Edge(node1, node2, 3.14)
dir12 = DirectedEdge(node1, node2)
wdir12_314 = DirectedEdge(node1, node2, 3.14)

e21 = Edge(node2, node1)
we21_314 = Edge(node2, node1, 3.14)
dir21 = DirectedEdge(node2, node1)
wdir21_314 = DirectedEdge(node2, node1, 3.14)

we12_3 = Edge(node1, node2, 3)
wdir12_3 = DirectedEdge(node1, node2, 3)

In [5]:
print(f"{e12 == e21}: {e12} equals {e21}")
print(f"{we12_314 == we21_314}: {we12_314} equals {we21_314}")
print(f"{dir12 == dir21}: {dir12} equals {dir21}")
print(f"{wdir12_314 == wdir21_314}: {wdir12_314} equals {wdir21_314}")
print(f"{we12_314 == we12_3}: {we12_314} equals {we12_3}")
print(f"{wdir12_314 == wdir12_3}: {wdir12_314} equals {wdir12_3}")

True: 1 <----> 2 equals 1 <----> 2
True: 1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2
False: 1 -----> 2 equals 2 -----> 1
False: 1 ---(3.14)--> 2 equals 2 ---(3.14)--> 1
False: 1 <--(3.14)--> 2 equals 1 <--(3)--> 2
False: 1 ---(3.14)--> 2 equals 1 ---(3)--> 2


In [6]:
print({e12, e21})  # exactly what we wanted because they are the same edge!
print({we12_314, we21_314})
print({node1, Node("1"), node2})

{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)}
{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=3.14)}
{Node(name='2'), Node(name='1')}


In [7]:
print(repr(DirectedEdge(node1, node2)))
print(repr(Edge(node1, node2)))

DirectedEdge(src=Node(name='1'), dest=Node(name='2'), weight=None)
Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)


In [8]:
try:
    Edge(node1, node2, -3.14)
except ValueError:
    print("test passed")

try:
    DirectedEdge(node1, node2, -3.14)
except ValueError:
    print("test passed")

test passed
test passed


# Digraphs and Graphs

In [20]:
class NWTuple(tuple):
    """
    Auxiliary node-weight tuple for Digraph and Graph.

    Saves space when weight=None and allows more readable downstream code.
    """

    # saves 8 bytes per weightless instance
    # ammortizes the memory cost of the class after about 100 wweightless instances
    def __new__(cls, node, weight=None):
        if weight is None:
            return super().__new__(cls, (node,))
        else:
            return super().__new__(cls, (node, weight))

    def __getitem__(self, index):  # type: ignore
        if index == 1 and len(self) == 1:
            return None
        return super().__getitem__(index)

    def __repr__(self):
        if len(self) == 1:
            return f"NWTuple({self[0]!r},)"
        else:
            return f"NWTuple({self[0]!r}, {self[1]!r})"

In [ ]:
class Digraph:
    """Directed graph."""

    # nodes is a list of the nodes in the graph.
    # edges is a dict mapping each node to a list of its children implemented
    # as node-weight tuples (NWTuples), where the first element is the child node
    # and the second element is the weight of the edge.
    # The first added edge (or the argument to __init__) determines whether
    # the Digraph is weighted for the rest of its life.
    # Internally, an "edge" is one-way only.
    # Bidirectional edges treated as having an "edge" in both directions.

    def __init__(self, weighted=None):
        """When weighted is None, it is determined by the first added edge."""
        self._nodes = []
        self._edges = {}
        self.weighted = weighted
        self._last_str = "Empty graph"
        self._last_str_valid = True

    def add_node(self, node: Node) -> None:
        if self.has_node(node):
            raise ValueError("Duplicate node")
        else:
            self._nodes.append(node)
            self._edges[node] = []
        self._last_str_valid = False

    def has_node(self, node: Node) -> bool:
        return node in self._nodes

    def _check_node_exists(self, node) -> None:
        if not self.has_node(node):
            raise ValueError("Node not in graph")

    def _check_weight_compatibility(self, edge) -> None:
        if edge.weighted and not self.weighted:
            raise ValueError("This graph is unweighted, cannot add weighted edge")
        if self.weighted and not edge.weighted:
            raise ValueError("This graph is weighted, cannot add unweighted edge")

    def _check_nodes_and_weights(self, edge) -> None:
        node1, node2 = edge.nodes
        self._check_node_exists(node1)
        self._check_node_exists(node2)
        if self.weighted is not None:
            self._check_weight_compatibility(edge)
        else:
            self.weighted = edge.weighted

    def _insert_edge(self, src, dest, weight) -> None:
        if dest in self.children_of(src):
            raise ValueError("Duplicate edge")
        self._edges[src].append(NWTuple(dest, weight))
        self._last_str_valid = False

    def add_directed_edge(self, edge) -> None:
        self._check_nodes_and_weights(edge)
        self._insert_edge(edge.src, edge.dest, edge.weight)

    def add_undirected_edge(self, edge) -> None:
        """Treated as two directed edges."""
        self._check_nodes_and_weights(edge)
        self._insert_edge(edge.node1, edge.node2, edge.weight)
        self._insert_edge(edge.node2, edge.node1, edge.weight)

    def children_of(self, node: Node) -> list:
        # never returns the list itself, just a copy
        self._check_node_exists(node)
        return [nwt[0] for nwt in self._edges[node]]  # nwt for node-weight tuple

    def _compute_str(self, cls) -> str:
        connected = set()
        edges_to_print = set()
        for node in self._nodes:
            for nwt in self._edges[node]:
                connected |= {node, nwt[0]}
                edges_to_print.add(cls(node, nwt[0], nwt[1]))
        unconnected = list(set(self._nodes) - connected)
        res = ""
        for edge in sorted(edges_to_print):
            res += f"{edge}\n"
        if len(unconnected) == 0:
            res += "No unconnected nodes."
        else:
            res += "Unconnected nodes:\n"
            for node in unconnected:
                res += f"{node}\n"
            res = res[:-1]
        return res

    def __str__(self) -> str:
        if self._last_str_valid:
            return self._last_str
        else:
            self._last_str = self._compute_str(DirectedEdge)
            self._last_str_valid = True
            return self._last_str


class Graph(Digraph):
    def add_directed_edge(self, edge):
        raise TypeError("Cannot add directed edge into undirected graph")

    def add_edge(self, edge):
        """Alias for add_undirected_edge"""
        self.add_undirected_edge(edge)

    def __str__(self) -> str:
        if self._last_str_valid:
            return self._last_str
        else:
            self._last_str = self._compute_str(Edge)
            self._last_str_valid = True
            return self._last_str

## Testing Digraphs and Graphs

In [21]:
edges = [(0, 1), (1, 2), (2, 3), (2, 4), (3, 4), (3, 5), (0, 2), (1, 0), (3, 1), (4, 0)]
nodes = []
for name in range(8):
    nodes.append(Node(str(name)))
digraph = Digraph()
digraph_w = Digraph(weighted=True)
graph = Graph()
graph_w = Graph()  # first edge will decide
for n in nodes:
    for g in [digraph, digraph_w, graph, graph_w]:
        g.add_node(n)
for edge in edges:
    digraph.add_directed_edge(DirectedEdge(Node(str(edge[0])), Node(str(edge[1]))))
    digraph_w.add_directed_edge(
        DirectedEdge(Node(str(edge[0])), Node(str(edge[1])), weight=round(random(), 2))
    )
    try:
        graph.add_edge(Edge(Node(str(edge[0])), Node(str(edge[1]))))
    except ValueError as m:
        if m == "Duplicate edge":
            pass
    try:
        graph_w.add_edge(
            Edge(Node(str(edge[0])), Node(str(edge[1])), weight=round(random(), 2))
        )
    except ValueError as m:
        if m == "Duplicate edge":
            pass

In [22]:
print(digraph)

0 -----> 1
0 -----> 2
1 -----> 0
1 -----> 2
2 -----> 3
2 -----> 4
3 -----> 1
3 -----> 4
3 -----> 5
4 -----> 0
Unconnected nodes:
6
7


In [23]:
print(digraph_w)

0 ---(0.68)--> 1
0 ---(0.83)--> 2
1 ---(0.59)--> 0
1 ---(0.58)--> 2
2 ---(0.17)--> 3
2 ---(0.03)--> 4
3 ---(0.41)--> 1
3 ---(0.35)--> 4
3 ---(0.15)--> 5
4 ---(0.32)--> 0
Unconnected nodes:
6
7


In [24]:
print(graph)

0 <----> 1
0 <----> 2
0 <----> 4
1 <----> 2
1 <----> 3
2 <----> 3
2 <----> 4
3 <----> 4
3 <----> 5
Unconnected nodes:
6
7


In [25]:
print(graph_w)

0 <--(0.22)--> 1
0 <--(0.8)--> 2
0 <--(0.04)--> 4
1 <--(0.41)--> 2
1 <--(0.68)--> 3
2 <--(0.87)--> 3
2 <--(0.32)--> 4
3 <--(0.08)--> 4
3 <--(0.9)--> 5
Unconnected nodes:
6
7


In [26]:
print(Graph())

Empty graph
